# Updated Workfow

Following from the draft workflow, I will now define several functions and generic scripts that will help segregate the input/output and the actual functions.

In [15]:
from pybedtools import BedTool # Note this requires bedtools
from glob import glob
import os
import pandas as pd
import numpy as np

In [2]:
# Ensure at top-level directory
os.chdir("../")
os.getcwd()

'/home/work2017/Documents/Jamin'

# Preprocessing in Bash

## liftOver

In [7]:
%%bash


# LIFTOVER FOR hg18 BED FILES -----------------------------------------------------------------------------
cd data/raw/markers/hg18
pwd

dir_chain=../../liftOverchains/hg18ToHg38.over.chain
dir_save=../../../preprocessed/markers_lifted/

for filename in *.bed;
do
    liftOver $filename $dir_chain $dir_save${filename/.bed/_lifted.bed} $dir_save${filename/.bed/_lifted.err}
done

/home/work2017/Documents/Jamin/data/raw/marks/hg18


Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates


> NOTE: ensure to keep hg18 files and hg19 files separated...

## Subsetting .maf Based on Clinical/Biospecimen Features

> Still to be done

## Converting .maf to .bed

In [10]:
%%bash

# Ensure data/ lies in your current directory
cd data/raw/mutations/

for filename in *.maf;
do 
    cut -f 5-8 $filename \
    | awk 'NR >= 6 {print}' \
    | awk '$3 = $3 FS "." FS "."' \
    | sed s/" "/"\t"/g \
    > ../../preprocessed/mutations_bed/${filename%.maf}.bed
done

# Populating the Blacklist for Filtering

> Still to be done...

# Intersection Function

> Note that this incorporates the filter step within it as well.

In [11]:
def bed_intersect(marker_bed, mutations_bed):
    # INPUTS:
    # marker_bed: BedTool object of the coordinates of the marker
    # mutations_bed: BedTool object of the coordinates of the mutations
    # OUTPUTS:
    # A tuple consisting of:
    # - fisher_test: object representing the output from bedtools fisher test (print to see)
    # - prop_intersection: % of marker_bed that intersected with mutations_bed
    # - intersect_bed: BedTool object of the intersections
    
    # Function set variables ------------------------------------------------------------------
    dir_blacklist = "data/raw/blacklists/"
    name_blacklist = "hg38.blacklist.bed"
    
    window_size = 1000 # size of windows around the points for intersection, if using window
    
    # Filtering -------------------------------------------------------------------------------
    
    # Produce blacklist BedTool instance
    blacklist = BedTool("".join([dir_blacklist, name_blacklist]))
    
    # Filter by subtracting blacklist from the original beds
    filtered_marker_bed = marker_bed.subtract(blacklist)
    filtered_mutations_bed = mutations_bed.subtract(blacklist)
    
    # Intersection ----------------------------------------------------------------------------
    
    # Intersection WITH WINDOW
    # intersect_bed = filtered_marker.window(filtered_mutations_bed, w=window_size)
    
    # Intersection WITHOUT WINDOW
    intersect_bed = filtered_marker.intersect(filtered_mutations_bed)
    
    # Statistics-------------------------------------------------------------------------------
    
    # Proportion of intersection
    prop_intersection = intersect_bed.count() / filtered_marker_bed.count()
    
    # For Fisher's exact test
    # Sorting first as bedtools documentations mentions that this is required
    # NOTE: seems that this is sorted such that chr12 comes before chr 2 - is this a problem? 
    # It seems that even the bedtools original would have done this too...
    sorted_marker_bed = filtered_marker_bed.sort()
    sorted_mutations_bed = filtered_mutations_bed.sort()
    fisher_test = sorted_marker_bed.fisher(sorted_mutations_bed, genome="hg38")
    
    # Output ----------------------------------------------------------------------------------
    
    return fisher_test, prop_intersection, intersect_bed    

There are some questions regarding the intersection that I have:

1. There is the possibility of using the window, but it appears this greatly increases the number of intersections. I wonder if the region of $\gamma$-H2AX is sufficient to include that of the mutations - is the window there in case of random shifts?
2. Does strandedness matter? I'm inclined to think not since it doesn't make sense to me how phosphorylated H2AX could be strand-selective, but there are both positive and negative strands in the bed file so I'm not entirely sure what that means...But then from visual inspection of the mutation data, they all seem positive...
3. There are some statistical tests already implemented in BedTools that might be useful: Fisher exact test seems most relevant, but the Jaccard statistics may be useful too. http://bedtools.readthedocs.io/en/latest/content/tools/fisher.html. I feel that unless there is some reason not to, this would not be a bad idea at the moment to use (at least until a better can be found)
4. See the sorting comment in the code above.

# Python Scripts

At this stage, we have done our preprocessing (producing a new set of BED files that can be used directly from Python with `pybedtools`, and need to stitch things together with the function created above.

The data we have preprocessed should be stored in the directories `data/preprocessed/marks_lifted` and `data/preprocessed/mutations_bed`. 

In [25]:
# INPUT/OUTPUT

dir_markers = "data/preprocessed/markers_lifted/"
dir_mutations = "data/preprocessed/mutations_bed/"

num_marker_files = sum(1 for file in glob("".join([dir_markers, "*.bed"])))
num_mutations_files = sum(1 for file in glob("".join([dir_mutations, "*.bed"])))

columns = ["marker_file", "mutation_file", "fisher_stat", "prop_intersection", "intersect_bed"]
num_rows = num_marker_files * num_mutations_files

agg_results = pd.DataFrame(np.nan, index=range(0, num_rows), columns=columns)

count = 0

# As always, need to be wary of the nested for loop...
for file_marker in glob("".join([dir_markers, "*.bed"])):
    print(file_marker)
    for file_mutations in glob("".join([dir_mutations, "*.bed"])):
        bed_marker = BedTool(file_marker)
        bed_mutations = BedTool(file_mutations)
        results = bed_intersect(bed_marker, bed_mutations)
        
        filename_marker = file_marker.replace(dir_markers, "")
        filename_mutations = file_mutations.replace(dir_mutations, "")
        
        agg_results.iloc[count] = [filename_marker, filename_mutations, results(0).two_tail, results(1), results(2)]

data/preprocessed/markers_lifted/GSM628531_cd4_h2ax_2_lifted.bed


NameError: name 'filtered_marker' is not defined